# Классификатор на KERAS

## Загружаем датасет

In [ ]:
import os
import numpy as np
import pandas as pd

data = pd.read_csv('big.csv',index_col=None, header=0, sep = '|')
data = data.dropna()


## Обучение модели

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers.core import Dense
from keras.layers import Conv1D, Input, Embedding
from keras.layers.pooling import GlobalMaxPooling1D
from keras.utils import to_categorical
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.initializers import lecun_uniform
from sklearn.model_selection import train_test_split
import pandas as pd

label_lst = sorted(data['label'].value_counts().index.to_list())
decoder = dict(enumerate(label_lst))
encoder = dict((j, i) for i, j in enumerate(label_lst))

X, Y = data['text'], to_categorical(data['label'].replace(encoder))
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)

max_words = 1000
keras_tokenizer = Tokenizer(num_words=max_words, char_level=False)
keras_tokenizer.fit_on_texts(X_train.tolist())

X_train = keras_tokenizer.texts_to_matrix(X_train, mode='tfidf')
X_val = keras_tokenizer.texts_to_matrix(X_val, mode='tfidf')

init = lecun_uniform(seed=42)
model = Sequential()
model.add(Embedding(10150, 100, input_length=1000))
model.add(Conv1D(128, 5, activation='relu', init=init))
model.add(GlobalMaxPooling1D())
model.add(Dense(80, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(4, activation='sigmoid'))
model.compile(
#     optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    epochs=15,
                    validation_data=(X_val, Y_val),
                    batch_size=64,
                    shuffle=True,
                    callbacks=[
                        ModelCheckpoint('model.hd5', save_best_only=True),
                        EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10),
                    ])
model.save_weights('weights.h5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 5, activation="relu", kernel_initializer=<keras.ini...)`


Train on 3196 samples, validate on 800 samples
Epoch 1/15
3196/3196 [==============================] - 1s 343us/step - loss: 0.4750 - acc: 0.7663 - val_loss: 0.2531 - val_acc: 0.8856
Epoch 2/15
3196/3196 [==============================] - 1s 217us/step - loss: 0.1688 - acc: 0.9506 - val_loss: 0.1023 - val_acc: 0.9697
Epoch 3/15
3196/3196 [==============================] - 1s 211us/step - loss: 0.1124 - acc: 0.9635 - val_loss: 0.0842 - val_acc: 0.9700
Epoch 4/15
3196/3196 [==============================] - 1s 219us/step - loss: 0.0945 - acc: 0.9681 - val_loss: 0.0906 - val_acc: 0.9647
Epoch 5/15
3196/3196 [==============================] - 1s 213us/step - loss: 0.0765 - acc: 0.9754 - val_loss: 0.0629 - val_acc: 0.9762
Epoch 6/15
3196/3196 [==============================] - 1s 211us/step - loss: 0.0676 - acc: 0.9767 - val_loss: 0.0508 - val_acc: 0.9800
Epoch 7/15
3196/3196 [==============================] - 1s 210us/step - loss: 0.0603 - acc: 0.9809 - val_loss: 0.0597 - val_acc: 0.9750
E

# Валидация модели

In [14]:
print ( model.summary())

decoder = {
    0: 'Сложный',
    1: 'Логичный',
    2: 'Простой',
    3: 'Спонтанный'
}

check = DataFrame({'text': [
 'Он свернулся в клубок и чуть шевелил во сне ушами. Вдруг за окном раздался шум. Проехала машина. Залаяла собака. Зачирикали птицы. Мурзик проснулся и потянулся. Потом он вспрыгнул на окно и стал смотреть вниз. Его очень интересует жизнь за окном. Люди любят смотреть телевизор. А Мурзик любит смотреть в окно. Это его телевизор. (60) У нас живёт кавказская овчарка. Это большой мохнатый пёс. Его зовут Арес. У него длинная белая шерсть. Он очень похож на белого медведя. Арес круглый год живёт на даче. Он сторожит дом. Пёс живёт в дощатой будке. Ему тепло на улице даже зимой. Арес отлич­ный сторож. Он не лает зря. Я часто гуляю с Аресом по полю и по лесу. Ведь ему нужно много двигаться. (66) Дети играли в квартире в прятки. Саша спрятался в шкафу за висящей одеждой. Дима спрятался на кухне за холодильником. Вера спряталась под столом за краем скатерти. Зина спряталась за открытой дверью. Гриша водил. Он сосчитал до ста. Потом открыл глаза и пошёл искать ребят. Первым он нашёл Зину. Потом нашёл Диму. Вера выскочила сама. А Сашу искали долго. Он заснул в шкафу. (63) Кирилл с папой долго гуляли по лесу. Мальчик увидел много разных деревьев. В лесу росли старые сосны и ели. Их высокие толстые стволы походили на колонны. Среди них стояли клёны и липы. Они были меньше хвойных деревьев. Кирилл всю прогулку слышал замечательный птичий концерт. Множество птиц щебетало и чирикало. А порой издалека доносился гулкий голос кукушки. (58) Среди морских рыб есть грозные хищники. Самые известные среди них акулы. Суще­ствует много разных видов акул. Есть акулы совсем небольшие. Их длина чуть больше метра. Они почти не опасны для человека. Но встречаются среди акул и гиганты. Это китовые акулы. Их длина достигает двадцати метров. Даже большие акулы могут быть безопасны для людей. Но всё равно. С любой акулой надо быть осторожным. (63) В холодных морях встречаются ледяные горы. Они называются айсбергами. Над водой видна только небольшая часть айсберга. Большая часть скрыта под водой. Айсберги явля­ются обломками ледников. Их лёд образовался на суше из снега. Поэтому он пресный. Из айсбергов моряки могут добывать пресную воду для питья. В айсберг стреляют из пушки. Потом садятся в шлюпку и подбирают осколки льда. (57) В Африке и в Южной Америке большие площади занимают саванны. Это равнины. Они покрыты высокой травянистой растительностью. Среди травы растут отдельные дере­вья. В саваннах часто встречается баобаб. В высоту он достигает двадцати пяти метров. А толщина ствола у него больше девяти метров. Баобаб может прожить больше четырёх тысяч лет. (50) Ангорская кошка является одной из самых старых пород кошек. Она происходит из Турции. Потом таких кошек стали разводить во Франции. У ангорской кошки длинная шерсть. Это небольшая и очень изящная кошка. У неё гибкое тело и стройные лапы. Ангор­ская кошка обладает дружелюбным и спокойным характером. Она умна и предана своему хозяину. Ангорская кошка любит привлекать к себе внимание человека и играть с ним. (65) Недавно в Америке вывели новую породу кошек. Она называется рэгдолл. Это крупная кошка. У неё широкая грудь и массивная задняя часть. Хвост у рэгдолла длинный и пуши­стый. Шея короткая и толстая. Лапы круглые и большие. Шерсть длинная и шелковистая. Внешне кошка кажется очень упитанной. Рэгдолл умеет очень хорошо расслабляться. Тогда его тело становится похожим на тряпичную куклу. (58) Дождевая и талая вода размывает землю. Образуются промоина. Она становится всё больше.',
 'Аналогичная тактика обороны аулов, основанная на их оставлении в случае опасности и отступлении в безопасное место,использовалась и чеченцами . Тогда как у дагестанцев в основе системы обороны аулов этот элемент совершенно отсутствовал: в случае опасности они не оставляли своих аулов, а основательно в них укрепившись, упорно оборонялись от неприятеля. Причинами различия оборонительных тактик у кавказских народов интересовались исследователи Кавказа непосредственно наблюдавшие ход Кавказской войны.В свое время данный вопрос поставил Н. Дубровин: «Почему те же черкесы очень редко защищали аул, тогда как жители Дагестана, напротив, оборонялись в своем селении слишком упорно? Объяснение этого явления можно найти только в особенностях быта обоих народов. Большая часть территории, населенной черкесским племенем, отличается плодородием, обилием леса и воды. Поэтому, если жена, дети и имущество были отправлены в горы или в безопасное место, то черкес легко кидал свою деревянную саклю и кусок обработанной им земли и без сожаления о них отправлялся далее в горы и в менее доступные места. При умеренности в пище и питье и при способности переносить все роды лишений,черкес знал, что и на новом месте найдет такой же хлеборобный кусок земли для посева кукурузы, его питающей; найдет в изобилии лес для постройки сакли и будет иметь такую же чистую воду и пастбище для быков. <...> Черкес защищался в ауле только в том случае, когда находились в опасности его жена, дети и имущество. Тогда он дрался с отчаянием и скорее сам погибал, нежели уступал что-либо врагу. Совсем в другом виде представляется быт дагестанского горца. Место, занимаемое жителями Дагестана, состоит по большей части из голых, безлесных и утесистых скал, песчаного или гранитного свойства; страна отличается недостатком воды и хороших плодородных земель. За неимением леса, горец строил свою хижину из камня; постройка ее стоила ему много труда, и потому он защищал свойаул от разорения» . Таким образом, при-родно-ландшафтный фактор и его социальное значение в данном случае был решающим.Рассмотрев различные варианты оборонительной тактики, применявшиеся черкесами, можно указать на то, что тактика обороны аулов была построена на базе природных условий, в которых они проживали. Данная тактика в сочетании с искусным ведением ими лесного и горного боя обладала высокой эффективностью. Во многом именно она была главной причиной того, что покорение черкесов в период Кавказской войны оставалось очень сложной задачей, требовавшей больших усилий и средств. Природные ландшафты сделались нерукотворными крепостями, частью военно-хозяйственного быта горцев, слитность с которым долго позволяла им противодействовать гигантской военно-экономической машине Российской империи. Российские военные власти и политическое руководство либо не оценили в должной мере природно-ландшафтного фактора в системе кавказского противостояния, либо с опозданием перешли к адаптации своих усилий к особенностям условий и тактики ведения войны в горно-лесистой местности.Императрица Екатерина II, Бецкий и их учреждения для воспитания русских женщин.',
 'В результате взрыва погибли по всей стране не готовы платить за последние годы переехали на данные партнера производителей электроники и другими прецедентными делами.За последние годы переехали на сайте московской полиции.), который был найден примерно в президенты США в Москву в феврале текущего года снижаются. Основным противником сирийских вооруженных сил США грозит до двух билетов на сайте коллектива, в ходе расследования и для него все будет принято, о том, что в феврале 2011 года, когда милиционеры его словам, последнюю неделю в том найдена ли он сможет превышать среднюю зарплату по облигациям, срок до сих пор не общался. Выступление Пауэлла в феврале текущего года в том числе кандидатов для того, как передает IsraelNN. По словам представителя КБ \"\"Малахит\"\", разработчика проекта 885 \"\"Ясень\"\" в ходе своего компьютера по всей стране, проведения массовых мероприятий. Об этом сообщает РИА \"\"Новости\"\". В нем будет создан специальный аналитический отдел по вопросам кинематографии, который был найден примерно через себя намного лучше?, ? сказал Петухов. На место в том же - подчеркнул Порошенко. В то же время в полицию сообщили, что в соответствии с начала года ? отметил автор фильма председателя правления РАО \"\"Газпром\"\" Алексея Миллера покинуть свой пост. При этом сообщает AFP. В результате обстрела областной думы\"\", - инопланетной расе из них ? сказал министр.Американской версии журнала Vogue не менее 10 тысяч рублей в результате обстрела областной администрации президента России Алексей Бабурин предупредил об этом сообщает The New York Times. В свою кандидатуру на сайте ежедневной американской рок-группы The Makeup Bullet имеет право проведения боя будет создан специальный аналитический отдел по его словам, негативно сказалось на сайте телеканала указано, что в ходе начавшейся поисковой операции по словам представителя КБ \"\"Малахит\"\", разработчика проекта постановления \"\"Об организации на сайте телеканала ТВС, найденные 17 октября. Своим названием институт обязан ученому Джеймсу Смитсону, который выступит в субботу серию многомесячных вооруженных столкновений между студентами и не менее 50 000 рублей инфляционных потерь, так как, по пассажирским и его словам, в том числе Романа Абрамовича, планирует провести проверку подобных медицинских учреждений по продаже здоровой еды.Российский миллиардер Сулейман Керимов занят поиском покупателя на сайте arXiv. \"\"состав эскадры достаточно внушителен - с ведущими мировыми центрами (Лондон, Гонконг). Ранее сообщалось, что в ходе избирательной компании. Об этом в районе станции на территории промышленного предприятия в том числе кандидато',
 'Доказательства вообще — Доказательство (Demonstratio) есть выведение истинности какого-либо положения (на основании силлогистических законов) из других положений. Доказывать можно лишь положения (понятия могут быть определяемы, факты — объясняемы и показываемы), и притом не все. Некоторые положения, обладающие самоочевидностью — аксиомы - не могут быть доказаны, а сами служат основанием всякого возможного Д. Иногда характер аксиомы приписывают положениям, которые не обладают самоочевидностью, но пользуются всеобщим признанием (напр. нравственные правила) и потому служат основой Д., хотя с точки зрения логической такое расширение понятия аксиомы недопустимо. Основанием для Д. может служить, кроме аксиом, еще и определение (напр. в юридических науках). Каждый волен дать определение понятия какое ему угодно, лишь бы оно не заключало в себе противоречия, и из него выводить следствия, которые должны считаться доказанными, если вывод сделан логически правильно, истинными же — лишь в той мере, в какой истинно определение, из которого они выведены. Факты сами по себе не могут служить основанием Д. (как, напр., учит Владиславлев в своей логике), а лишь инстанциями, из которых составляется довод в пользу того или иного положения. Многие положения хотя и могли бы быть доказаны, но за отсутствием доводов, т. е. тех понятий (средних терминов), которые могли бы связать эти положения с самоочевидными истинами, до поры до времени считаются недоказанными и выражаются условно (см. Гипотеза). Нет основания отличать опровержение от Д., ибо опровержение есть Д. ложности известного положения. Опровержение состоит в том, что мы показываем ложность посылок, из которых были делаемы заключения, или показываем ложность самого заключения, или, наконец, ослабляем значение доводов, т. е. стараемся показать, что приводимые понятия и факты не могут играть роли среднего термина. Отношение между доказываемым положением (проблемой, теоремой) и самим Д. более сложно, чем обычно предполагают. Всякое положение допускает различные Д. своей истинности, хотя не все они одинаково сильны и убедительны; поэтому опровержение Д. не должно непременно влечь за собой и опровержение самого положения. В истине два момента: объективный — соответствие познания познаваемому предмету, и субъективный — уверенность, что познание действительно соответствует познаваемому предмету. Д. должно иметь в виду оба момента — как истинность, так и уверенность в ней. Если Д. стремится лишь вызвать убеждение в истинности, не обращая внимания на саму истинность, то такое Д. становится софистическим, а искусство вызывать уверенность в истине одного положения и уничтожать уверенность противоположного свойства называется эристикой. Характер Д. и степень его точности зависят от положения, к которому оно применяется. В этом отношении, напр., Евклидовой геометрии был делаем справедливый упрек, что она заботится более о том, чтобы вынудить признание истинности какой-либо теоремы путем чистой логики, чем о том, чтобы сделать свои Д. очевидными и наглядными, каким характером должна отличаться геометрия. — Из всего сказанного вообще о доказательствах следует заключить, что: 1) знание возможно вне Д.; 2) что Д. не дает нам вполне новых знаний, но 3) придает знанию характер достоверности.'
], 'label': [[],[],[],[]]
})

tokenized_check = keras_tokenizer.texts_to_matrix(check['text'], mode='tfidf') 

check['label'] = pd.Series(np.argmax(model.predict(tokenized_check), axis=1)).replace(decoder)
check['weights'] = [str(row) for row in tokenized_check] #pd.Series(model.predict(tokenized_check))

print (check)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1000, 100)         1015000   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 80)                10320     
_________________________________________________________________
dense_5 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 124       
Total params: 1,092,002
Trainable params: 1,092,002
Non-trainable params: 0
____________________________________________